# Tutorial configuration

In [ ]:
!rm -rf snapearthapis
!git clone https://github.com/chicham/snapearthapis.git
!pip install -r snapearthapis/requirements.txt
!!add-apt-repository ppa:longsleep/golang-backports -y
!apt-get update > /dev/null && apt-get -y install jq golang-go > /dev/null
!go version
!!go install github.com/fullstorydev/grpcurl/cmd/grpcurl@latest
import os
os.environ['PATH'] += ":/root/go/bin"

Cloning into 'snapearthapis'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 177 (delta 68), reused 157 (delta 48), pack-reused 0
Receiving objects: 100% (177/177), 48.53 KiB | 2.21 MiB/s, done.
Resolving deltas: 100% (68/68), done.
     |████████████████████████████████| 96 kB 3.4 MB/s 
     |████████████████████████████████| 95 kB 3.1 MB/s 
     |████████████████████████████████| 61 kB 6.6 MB/s 
     |████████████████████████████████| 133 kB 51.0 MB/s 
     |████████████████████████████████| 253 kB 42.0 MB/s 
     |████████████████████████████████| 19.3 MB 1.6 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 2.0 MB 35.9 MB/s 
     |████████████████████████████████| 138 kB 39.7 MB/s 
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.0.1
    Uninstalling MarkupSafe-2.0.1:
      Successfully uninst

# EarthSignature Tutorial

## Introduction

The goal of this notebook is to show how to use the EarthSignature API to download segmented Sentinel2 products. At the end of this tutorial, you will be able to query the API, understand how to filter the results when querying the API and understand what are the data returned by the API.


## Visualize the products available

In [ ]:
import orjson as json 
lines = !grpcurl -insecure  -max-msg-sz 1073741824 earthsignature.snapearth.eu:443 snapearth.api.v1.database.DatabaseProductService.ListProductIds | jq -c -M
products = [json.loads(line) for line in lines]

In [ ]:
import base64

import folium
from shapely import wkt
from shapely.geometry import mapping


EUROPE_COORDINATES = wkt.loads(
    "POLYGON((-10.61 71.16, 44.85 71.16, 44.85 35.97, -10.61 35.97, -10.61 71.16))",
)
MAP_CENTER = mapping(EUROPE_COORDINATES.centroid)

In [ ]:

map_ = folium.Map(
    location=MAP_CENTER["coordinates"][::-1],
    zoom_start=4,
    crs="EPSG3857",
)


for product in products:
    geom = wkt.loads(product['wkt'])
    folium.GeoJson(data=geom).add_to(map_)
    centroid = mapping(geom.centroid)
    folium.Marker(
            location=centroid["coordinates"][::-1],
            tooltip=f'product["productId"]',
        ).add_to(map_)    

map_


## Querying the API


In [1]:
# We delete old results first
!rm -rf api_results
!mkdir api_results
!grpcurl -max-msg-sz 1073741824 -d '{"n_results": 5, "start_date": "2021-01-01T00:00:00Z", "end_date": "2022-05-01T00:00:00Z"}' -insecure earthsignature.snapearth.eu:443 snapearth.api.v1.database.DatabaseProductService.ListSegmentation | jq -c -M '{productId: .productId, segmentation: .segmentation, cloudMask: .cloudMask}' > api_results/responses.jsonl

from pathlib import Path
import orjson as json

with Path('api_results/responses.jsonl') as src:
  for line in src:
    print('toto')

/bin/bash: jq: command not found
/bin/bash: grpcurl: command not found


ModuleNotFoundError: ignored